# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [11]:
import nltk
nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import pickle

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier

In [4]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse_AD.db')
df = pd.read_sql_table("Message", con=engine)
X = df.message.values
Y = df.drop(columns=["id", "message", "original", "genre"]).values

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

In [12]:
tokenize("aplusk Breaking news Catastrophic earthquake today in Haiti just off the coast of Florida. Untold dead. Horrific situ. PLEEZ get word out")

['aplusk',
 'breaking',
 'news',
 'catastrophic',
 'earthquake',
 'today',
 'in',
 'haiti',
 'just',
 'off',
 'the',
 'coast',
 'of',
 'florida',
 '.',
 'untold',
 'dead',
 '.',
 'horrific',
 'situ',
 '.',
 'pleez',
 'get',
 'word',
 'out']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [39]:
pipeline = Pipeline([
        ("vect", CountVectorizer(tokenizer=tokenize)),
        ("tfidf", TfidfTransformer()),
        ("clf", MultiOutputClassifier(RandomForestClassifier())),
    ])

pipeline

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [40]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [24]:
out_cols = df.drop(columns=["id", "message", "original", "genre"]).columns
out_cols

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [41]:
Y_pred = pipeline.predict(X_test)

In [42]:
for i, col in enumerate(out_cols):
    print("\n\n#########\n\n")
    print("{0}: {1}\n".format(i, col))
    print(classification_report(Y_test[i], Y_pred[i]))



#########


0: related

             precision    recall  f1-score   support

          0       0.80      1.00      0.89        28
          1       1.00      0.12      0.22         8

avg / total       0.84      0.81      0.74        36



#########


1: request

             precision    recall  f1-score   support

          0       0.69      1.00      0.81        24
          1       1.00      0.08      0.15        12

avg / total       0.79      0.69      0.59        36



#########


2: offer

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        36

avg / total       1.00      1.00      1.00        36



#########


3: aid_related

             precision    recall  f1-score   support

          0       0.94      1.00      0.97        33
          1       1.00      0.33      0.50         3

avg / total       0.95      0.94      0.93        36



#########


4: medical_help

             precision    recall  f1-score   support

  

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [44]:
parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__max_features': (None, 5000, 10000),
    'tfidf__use_idf': (True, False),
#     'clf__n_estimators': [50, 100, 200],
#     'clf__min_samples_split': [2, 3, 4],
}

parameters

{'vect__ngram_range': ((1, 1), (1, 2)),
 'vect__max_df': (0.5, 0.75, 1.0),
 'vect__max_features': (None, 5000, 10000),
 'tfidf__use_idf': (True, False)}

In [45]:
cv = GridSearchCV(pipeline, param_grid=parameters)
cv.fit(X_train, Y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'vect__ngram_range': ((1, 1), (1, 2)), 'vect__max_df': (0.5, 0.75, 1.0), 'vect__max_features': (None, 5000, 10000), 'tfidf__use_idf': (True, False)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [46]:
Y_pred_cv = cv.predict(X_test)

In [47]:
for i, col in enumerate(out_cols):
    print("\n\n#########\n\n")
    print("{0}: {1}\n".format(i, col))
    print(classification_report(Y_test[i], Y_pred_cv[i]))



#########


0: related

             precision    recall  f1-score   support

          0       0.85      1.00      0.92        28
          1       1.00      0.38      0.55         8

avg / total       0.88      0.86      0.84        36



#########


1: request

             precision    recall  f1-score   support

          0       0.67      1.00      0.80        24
          1       0.00      0.00      0.00        12

avg / total       0.44      0.67      0.53        36



#########


2: offer

             precision    recall  f1-score   support

          0       1.00      0.97      0.99        36
          1       0.00      0.00      0.00         0

avg / total       1.00      0.97      0.99        36



#########


3: aid_related

             precision    recall  f1-score   support

          0       0.94      1.00      0.97        33
          1       1.00      0.33      0.50         3

avg / total       0.95      0.94      0.93        36



#########


4: medical_help

   

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
with open("classifier.pkl", "wb") as outfile:
    pickle.dump(pipeline, outfile)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.